In [1]:
import numpy as np
import pandas as pd
import math as math
import sys

In [2]:
from scipy.spatial import ConvexHull, convex_hull_plot_2d

In [3]:
def minBoundingRect(hull_points_2d):
    #print "Input convex hull points: "
    #print hull_points_2d

    # Compute edges (x2-x1,y2-y1)
    edges = np.zeros( (len(hull_points_2d)-1,2) ) # empty 2 column array
    for i in range( len(edges) ):
        edge_x = hull_points_2d[i+1,0] - hull_points_2d[i,0]
        edge_y = hull_points_2d[i+1,1] - hull_points_2d[i,1]
        edges[i] = [edge_x,edge_y]
    #print "Edges: \n", edges

    # Calculate edge angles   atan2(y/x)
    edge_angles = np.zeros( (len(edges)) ) # empty 1 column array
    for i in range( len(edge_angles) ):
        edge_angles[i] = math.atan2( edges[i,1], edges[i,0] )
    #print "Edge angles: \n", edge_angles

    # Check for angles in 1st quadrant
    for i in range( len(edge_angles) ):
        edge_angles[i] = abs( edge_angles[i] % (math.pi/2) ) # want strictly positive answers
    #print "Edge angles in 1st Quadrant: \n", edge_angles

    # Remove duplicate angles
    edge_angles = np.unique(edge_angles)
    #print "Unique edge angles: \n", edge_angles

    # Test each angle to find bounding box with smallest area
    min_bbox = (0, sys.maxsize, 0, 0, 0, 0, 0, 0) # rot_angle, area, width, height, min_x, max_x, min_y, max_y
   
    for i in range( len(edge_angles) ):

        # Create rotation matrix to shift points to baseline
        # R = [ cos(theta)      , cos(theta-PI/2)
        #       cos(theta+PI/2) , cos(theta)     ]
        R = np.array([ [ math.cos(edge_angles[i]), math.cos(edge_angles[i]-(math.pi/2)) ], [ math.cos(edge_angles[i]+(math.pi/2)), math.cos(edge_angles[i]) ] ])
        #print "Rotation matrix for ", edge_angles[i], " is \n", R

        # Apply this rotation to convex hull points
        rot_points = np.dot(R, np.transpose(hull_points_2d) ) # 2x2 * 2xn
        #print "Rotated hull points are \n", rot_points

        # Find min/max x,y points
        min_x = np.nanmin(rot_points[0], axis=0)
        max_x = np.nanmax(rot_points[0], axis=0)
        min_y = np.nanmin(rot_points[1], axis=0)
        max_y = np.nanmax(rot_points[1], axis=0)
        #print "Min x:", min_x, " Max x: ", max_x, "   Min y:", min_y, " Max y: ", max_y

        # Calculate height/width/area of this bounding rectangle
        width = max_x - min_x
        height = max_y - min_y
        area = width*height
        #print("Potential bounding box ", i, ":  width: ", width, " height: ", height, "  area: ", area)

        # Store the smallest rect found first (a simple convex hull might have 2 answers with same area)
        if (area < min_bbox[1]):
            min_bbox = ( edge_angles[i], area, width, height, min_x, max_x, min_y, max_y )
        # Bypass, return the last found rect
        #min_bbox = ( edge_angles[i], area, width, height, min_x, max_x, min_y, max_y )

    # Re-create rotation matrix for smallest rect
    angle = min_bbox[0]   
    R = np.array([ [ math.cos(angle), math.cos(angle-(math.pi/2)) ], [ math.cos(angle+(math.pi/2)), math.cos(angle) ] ])
    #print "Projection matrix: \n", R

    # Project convex hull points onto rotated frame
    proj_points = np.dot(R, np.transpose(hull_points_2d) ) # 2x2 * 2xn
    #print "Project hull points are \n", proj_points

    # min/max x,y points are against baseline
    min_x = min_bbox[4]
    max_x = min_bbox[5]
    min_y = min_bbox[6]
    max_y = min_bbox[7]
    #print "Min x:", min_x, " Max x: ", max_x, "   Min y:", min_y, " Max y: ", max_y

    # Calculate center point and project onto rotated frame
    center_x = (min_x + max_x)/2
    center_y = (min_y + max_y)/2
    center_point = np.dot( [ center_x, center_y ], R )
    #print "Bounding box center point: \n", center_point

    # Calculate corner points and project onto rotated frame
    corner_points = np.zeros( (4,2) ) # empty 2 column array
    corner_points[0] = np.dot( [ max_x, min_y ], R )
    corner_points[1] = np.dot( [ min_x, min_y ], R )
    corner_points[2] = np.dot( [ min_x, max_y ], R )
    corner_points[3] = np.dot( [ max_x, max_y ], R )
    #print "Bounding box corner points: \n", corner_points

    #print "Angle of rotation: ", angle, "rad  ", angle * (180/math.pi), "deg"

    return (angle, min_bbox[1], min_bbox[2], min_bbox[3], center_point, corner_points) # rot_angle, area, width, height, center_point, corner_points



In [4]:
bucket_size = 10
valori_ts = []
classi = []

path = ""

# importiamo il dataset

with open(path + 'ThingspeakEU.meta.csv', 'r', encoding='utf-8') as dati:
    for row in dati:
        riga = row.strip().split(',')
        classe = int(riga[8])
        classi.append(classe)
        valori = np.array(riga[9:]).astype(np.float)
        valori_ts.append(valori)

In [110]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler, MaxAbsScaler, Normalizer, StandardScaler

X = Normalizer().fit_transform(valori_ts)
X = MinMaxScaler(feature_range=(0, 96)).fit_transform(valori_ts)

In [6]:
# aggiunge bucket ad S
def buckets(S, ts, bucket_size, index):
    # num of points, beg, end, l, r
    S.append([1, index, index, ts[index], ts[index]])
    if (len(S) > bucket_size):
        # indice del bucket che unito al successivo produce minore errore
        i = findMin(S, ts)
        b = union(S[i], S[i+1], ts)
        successor = S[i+1]
        S.remove(S[i])
        S.remove(successor)
        S.insert(i, b)
    return S

In [7]:
# calcola il massimo errore all'interno del bucket
def compute_error(b, ts):
    if (b[0] == 1 or b[0] == 2):
        return 0
    else:
        maxErr = 0
        for i in range(b[1], b[2]+1):
            # valore approssimato = intercept + index*slope
            v = b[6] + i*b[5]
            err = abs(ts[i]-v)
            
            if (err > maxErr):
                maxErr = err
    
    return maxErr

In [8]:
# trova indice del bucket che unito al successivo produce minore errore
def findMin(S, ts):
    minErr = float('Inf')
    index = None
    for i in range(len(S)-1):
        err = compute_error(S[i], ts)
        union_error = compute_error(union(S[i], S[i+1], ts), ts)
        
        if (union_error - err < minErr or union_error - err == 0):
            minErr = union_error - err
            index = i
    
    return index

In [9]:
# unisce due bucket
def union(b1, b2, ts):
    beg = b1[1]
    end = b2[2]

    # ciascun bucket ha un punto solo
    if (b1[0] == 1 and b2[0] == 1):
        l = b1[3]
        r = b2[3]
        if (l == r):
            i = l
            s = 0
        else:
            s = slope([b1[1], b1[3]], [b2[1], b2[3]])
            i = intercept([b1[1], b1[3]], [b2[1], b2[3]])
        return [2, beg, end, l, r, s, i]
    
    else:
        points = []
        
        # raggruppiamo tutti i punti dei due bucket
        for i in range(b1[1], b1[2]+1):
            points.append([i, ts[i]])
        for j in range(b2[1], b2[2]+1):
            points.append([j, ts[j]])
        
        # controlliamo che i punti non siano orizzontali
        unique = 0
        for point in points:
            if (abs(points[0][1] - point[1]) > 3.1e-13):
                unique = unique + 1
        if (unique == 0):
            l = b1[3]
            r = b2[3]
            s = 0
            return [len(points), beg, end, l, r, s, l]
        
        # controlliamo che i punti non si trovino sulla stessa retta
        slope_check = slope(points[0], points[1])
        for k in range(2, len(points)):
            s = slope(points[0], points[k])
            if (abs(s - slope_check) > 3.1e-13):
                break
            if (k == len(points)-1):
                l = b1[3]
                r = b2[4]
                i = intercept(points[0], points[1])
                return [len(points), beg, end, l, r, s, i]
        
        # calcoliamo convex hull
        h = ConvexHull(points)

        hull_vertices = []

        for vertice in h.vertices:
            hull_vertices.append(points[vertice])

        bbox = minBoundingRect(np.array(hull_vertices))
        
        # punti medi dei vertici del box
        vertices = bbox[5]
        m1 = [(vertices[0][0] + vertices[1][0])/2, (vertices[0][1] + vertices[1][1])/2]
        m2 = [(vertices[2][0] + vertices[3][0])/2, (vertices[2][1] + vertices[3][1])/2]
        m3 = [(vertices[0][0] + vertices[3][0])/2, (vertices[0][1] + vertices[3][1])/2]
        m4 = [(vertices[1][0] + vertices[2][0])/2, (vertices[1][1] + vertices[2][1])/2]
        
        # prendiamo il lato più piccolo del box 
        if (distance(m3, m4) > distance(m1, m2)):
            m1 = m3
            m2 = m4
            
        #plt.plot([point[0] for point in vertices], [point[1] for point in vertices])
        #plt.plot([point[0] for point in points], [point[1] for point in points])
        #plt.plot(m1[0], m1[1], 'ro')
        #plt.plot(m2[0], m2[1], 'ro')
        #plt.show()
        
        # la bisettrice del box ha pendenza uguale alla retta che passa dai punti medi dei lati del box
        s = slope(m1, m2)
        i = intercept(m1, m2)
        l = s*beg + i
        r = s*end + i

        return [len(points), beg, end, l, r, s, i]

In [10]:
# calcolo pendenza/coefficiente angolare tra due punti
def slope(p1, p2):
    slope = (p2[1]-p1[1])/(p2[0]-p1[0])
    return slope

In [11]:
# calcolo intercetta della retta tra due punti
def intercept(p1, p2):
    intercept = (p1[0]*p2[1] - p2[0]*p1[1])/(p1[0]-p2[0])
    return intercept

In [12]:
# distanza tra due punti
def distance(p1, p2):
    d = math.sqrt((p2[0]-p1[0])**2 + (p2[1] - p2[1])**2)
    return d

In [13]:
# trasformiamo le pendenze in gradi e li assegniamo ad un carattere
def symbolization(S):
    
    d = {'a' : 0, 'b' : 0, 'c' : 0, 'd' : 0}
    angles = []
    for b in S:
        if (b[0] != 1):
            ang = math.degrees(math.atan(b[5]))
            if (ang < 90 and ang >= 45):
                d['a'] = d['a'] + 1
            elif (ang < 45 and ang >= 0):
                d['b'] = d['b'] + 1
            elif (ang < 0 and ang >= -45):
                d['c'] = d['c'] + 1
            elif (ang < -45):
                d['d'] = d['d'] + 1
    
    return d

In [111]:
slopes = []

for ts in X:
    S = []
    for i in range(len(ts)):
        S = buckets(S, ts, bucket_size, i)
    slopes.append(S)

In [112]:
ds = []

for S in slopes:
    ds.append(symbolization(S))

In [113]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

In [114]:
def metrics(y_pred, y_test):
    accuracy = accuracy_score(y_pred, y_test)*100
    f1 = f1_score(y_pred, y_test, average = 'macro')*100
    return [accuracy, f1]

In [115]:
# creiamo il dataframe
data = pd.DataFrame(ds)
X_train, X_test, y_train, y_test = train_test_split(data, classi, test_size = 0.3, random_state = 100)

In [156]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_pred, y_test)
f1_score(y_pred, y_test, average='macro')

C:\Users\Deggg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.054878465391964795

In [ ]:
# Standard Thingspeak k=26 [0.296703, 0.102272]
# Normalizer Thingspeak k=24 [0.194662, 0.045880]
# MinMax Thingspeak k=14 [0.240188, 0.048623]